In [ ]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
import os

# Pfad zum Ordner mit den Bildern
path_to_images_folder = '<Saalbach Bilder>'
# Pfad zum Speichern der Ergebnisse
path_to_results = '<CSV-Dateiname'
# Pfad zur Yolo-Datei
model_file = '<YOLO Modell>'

# Initialisierung des Modells mit dem Pfad zur trainierten Modell-Datei
model = YOLO(model_file)

# Verarbeitung aller JPEG-Bilder im angegebenen Verzeichnis und Speicherung der Ergebnisse
results = model.track(path_to_images_folder, show=True, conf=0.60) 

# Funktion, um zu verhindern, dass der Index in die CSV geschrieben wird
def new_func():
    return False

# Verarbeitung jedes Ergebnisses, um Keypoints und Tracking-Daten zu verarbeiten
for frame_number, r in enumerate(results):
        
    for detection in r:
                
        # Annahme, dass person_id korrekt extrahiert wird
        person_id = int(detection.boxes.id.numpy()[0])  # Annahme, dass person_id ein Skalar ist
        # Annahme, dass Keypoints auf der GPU sind und auf die CPU übertragen und umgeformt werden müssen
        kp = detection.keypoints.xy.cpu().numpy()
        reshaped_kp = kp.reshape((17, 2))
        
        print(reshaped_kp)
        print(f"Frame: {frame_number}, Person ID: {person_id}")
        
        # Erstellung eines DataFrame mit Keypoints, Frame-Nummer und Person-ID
        df = pd.DataFrame(reshaped_kp, columns=['x', 'y'])
        df['frame_number'] = np.full((17,), frame_number)  # Wiederholung der Frame-Nummer für jeden Keypoint
        df['person_id'] = np.full((17,), person_id)  # Wiederholung der Person-ID für jeden Keypoint
        
        # Anhängen des DataFrame an eine CSV-Datei
        csv_path = path_to_results
        df.to_csv(csv_path, index=new_func(), mode='a', header=not os.path.exists(csv_path))
        
        # Optional: Druck des CSV-Strings zur Überprüfung der Ausgabe
        csv_data = df.to_csv()
        print('\nCSV String:\n', csv_data)
